In [1]:
DATA_DIR = '/lustre/groups/ml01/workspace/louis.kuemmerle/projects/A1/data2/' # "./../"
DATA_VERSION = 'oct22'
RESULTS_DIR = '/lustre/groups/ml01/workspace/louis.kuemmerle/projects/A1/results/'
SHAM = True

########################################################################################
sham_str = '_wSham' if SHAM else ''

In [2]:
import pandas as pd
import scanpy as sc
import matplotlib.pyplot as plt

In [3]:
samples = [
    'MUC12819',
    'MUC12820',
    'MUC12821',
    'MUC12822',
    'MUC12823',
    'MUC12824',
    'MUC12825',
    'MUC12826',
    'MUC12827',
    'MUC12828',
    'MUC12829',
    'MUC12830',
    'MUC12831',
    'MUC12832',
    'MUC12833',
    'MUC12834',
    'MUC12835',
    'MUC12836',
    'MUC12837',
    'MUC12838',
    'MUC12839',
    'MUC12840',
    'MUC12841',
    'MUC12842',
    'MUC12843',
    'MUC12844',
    'MUC12845',
    'MUC12846',
    'MUC12847',
    'MUC12848',
    'MUC12849',
    'MUC12850'
]

# Stacked bar plots of cell type proportions 
(erythroid cells are not included since they messed up the deconvolution)

In [4]:
adata = sc.read(DATA_DIR+f'cellxgene_{DATA_VERSION}{sham_str}_umaps.h5ad')

In [5]:
# Get donor deconvolution info
dfs = []
for i,sample in enumerate(samples):
    df = pd.read_csv(RESULTS_DIR+f"donor_deconv/vireo_noeryt/{sample}/donor_ids.tsv", sep='\t')
    df.index = [idx+f"-{i}" for idx in df["cell"].values]
    df["sample_donor_id"] = sample + "_" + df["donor_id"]
    df = df[["donor_id","sample_donor_id"]]
    dfs.append(df)
df = pd.concat(dfs)

# A few cells were filtered when updating annotations:
df = df.loc[df.index.isin(adata.obs_names)]

# Add info to adata

adata = adata[df.index]
adata.obs["donor_id"] = df["donor_id"]
adata.obs["sample_donor_id"] = df["sample_donor_id"]

# filter out Meninges and Brain + unassigned and doublet cells from deconvolution
bones = ['Femur', 'Humerus', 'Pelvis', 'Scapula', 'Skull', 'Vertebra']
adata = adata[adata.obs["region"].isin(bones)]
adata = adata[adata.obs["donor_id"].isin(["donor0","donor1","donor2"])]

# Create column for plot
rename_bones = {
    "Skull":"Calvaria",
    "Pelvis":"Pelvis",
    "Femur":"Femur",
    "Shoulder":"Scapula",
    "Forearm":"Humerus",
    "Vertebra":"Vertebra",
    #"Meninges":"Meninges",
    #"Brain":"Brain"
}
rename_donor = {"donor0":"1","donor1":"2","donor2":"3"}#,"unassigned":"4","doublet":"5"}
adata.obs["region_sample_donor"] = adata.obs[["sample_name","donor_id"]].apply(lambda s: rename_bones[s[0].split("_")[0]] + "_" + s[0].split("_")[2] + "." + rename_donor[s[1]],axis=1)

/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


In [6]:
%load_ext rpy2.ipython

In [7]:
%%R
library(ggplot2)

(Execute cells till here also for level2 plots)

# Sham vs MCAO comparison now additionally with one bar of Naive

#### function for plot df

In [8]:
def get_proportions_df(adata,condition=None):
    """
    """
    
    a = adata
    if condition:
        a = a[a.obs["condition"]==condition]
    
    df = pd.crosstab(a.obs["region_sample_donor"],a.obs["level1"]).copy()
    df = df.div(df.sum(axis=1), axis=0)

    df.index = df.index.astype(str)
    df.columns = df.columns.astype(str)
    df = df.reset_index()
    df = df.melt(id_vars='region_sample_donor')
    df = df.rename(columns={'value': 'perc', 'level1': 'celltype'})
    if condition:
        df["condition"] = condition
    
    return df

#### prepare variables for plot

In [9]:
# Order for regions and conditions
cond_order = ["Naive","Sham","MCAo"]
#region_order = ["Calvaria","Vertebra","Scapula","Humerus","Pelvis","Femur"]
region_sample_donor_order = [
    "Calvaria_1.1","Calvaria_1.2","Calvaria_1.3","Calvaria_2.1","Calvaria_2.2","Calvaria_2.3",
    "Vertebra_1.1","Vertebra_1.2","Vertebra_1.3","Vertebra_2.1","Vertebra_2.2","Vertebra_2.3",
    "Scapula_1.1","Scapula_1.2","Scapula_1.3","Scapula_2.1","Scapula_2.2","Scapula_2.3",
    "Humerus_1.1","Humerus_1.2","Humerus_1.3","Humerus_2.1","Humerus_2.2","Humerus_2.3",
    "Pelvis_1.1","Pelvis_1.2","Pelvis_1.3","Pelvis_2.1","Pelvis_2.2","Pelvis_2.3",
    "Femur_1.1","Femur_1.2","Femur_1.3","Femur_2.1","Femur_2.2","Femur_2.3",
]
ct_order = [
    'progenitors', 'neutrophil', 'monocyte', 'B cell', 'T cell', 'NK cell',
    'NK-T cell', 'dendritic cell', 'macrophage', #'microglia',
    'basophil', 'structural cell', #'erythroid precursor', 'erythroid cell', 
    'brain cell', 'megakaryocyte', 'innate lymphoid cell'
]
ct_colors = adata.uns["level1_colors"].tolist()

# Get df
#bones = ['Femur', 'Humerus', 'Pelvis', 'Scapula', 'Skull','Vertebra']
dfs = [
    get_proportions_df(adata,condition="Naive"),
    get_proportions_df(adata,condition="Sham"),
    get_proportions_df(adata,condition="MCAO"),
]

df = pd.concat(dfs)
df.loc[df["condition"]=="MCAO","condition"] = "MCAo"
#df.loc[df["region"]=="Skull","region"] = "Calvaria"

df["condition"] = df["condition"].astype("category")
df["condition"] = df["condition"].cat.reorder_categories(cond_order)
#df["region"] = df["region"].astype("category")
#df["region"] = df["region"].cat.reorder_categories(region_order)
df["region_sample_donor"] = df["region_sample_donor"].astype("category")
df["region_sample_donor"] = df["region_sample_donor"].cat.reorder_categories(region_sample_donor_order)
df["celltype"] = df["celltype"].astype("category")
df["celltype"] = df["celltype"].cat.reorder_categories(ct_order)#[::-1])
df["perc"] *= 100

/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)


#### stacked bar plot
To see the plot example comment out `ggsave`. Note however that the figure size differs between jupyter output and saved pdf

In [10]:
%%R -i df -i ct_colors

#font sizes
S_SIZE = 17
M_SIZE = 19
L_SIZE = 21

#figure size
WIDTH=10#4.33
HEIGTH=10

# Labels
X_LABEL = 'Region_Sample.Donor'
Y_LABEL = 'Cell proportion'
LEGEND_LABEL = 'Cell types'

options(repr.plot.width=1, repr.plot.height=1)
g1 <- ggplot(df, aes(x=region_sample_donor, y=perc, fill=celltype)) +
    scale_fill_manual(values = ct_colors) +
    geom_bar(position='stack',stat='identity')  +  
    labs(x=X_LABEL, y=Y_LABEL) +
    facet_grid(~condition, scales="free", space="free_x") +
    theme_classic() + 
    guides(fill=FALSE)+#guide_legend(title=LEGEND_LABEL,keyheight=0.8,keywidth=0.8,default.unit="cm")) +
    theme(strip.text = element_text(face="bold", size=L_SIZE),   # size of subtitles ("MCAO", "Naive")
          axis.title.x = element_text(face="bold", size=M_SIZE), # x axis label (Cell type)
          axis.title.y = element_text(face="bold", size=M_SIZE), # y axis label (Cell proportion)
          axis.text.x  = element_text(angle=90, hjust = 1, vjust =0.5,  size=S_SIZE-6), # x axis ticks (regions)
          axis.text.y = element_text(size=S_SIZE), # y axis tick labels (cell proportion percentage)
          legend.title = element_text(size=M_SIZE),
          legend.text = element_text(size=S_SIZE))
g1
ggsave(filename = paste0('./figures/stacked_proportions_donor_deconv.pdf'), 
      plot = g1, width = WIDTH, height=HEIGTH)

## Same plots for level2 subsets

#### function for plot df

In [11]:
def get_proportions_df(adata,condition=None,celltypes=None):
    """
    """
    
    a = adata
    if condition:
        a = a[a.obs["condition"]==condition]
    if celltypes:
        a = a[a.obs["level2"].isin(celltypes)]
    
    df = pd.crosstab(a.obs["region_sample_donor"],a.obs["level2"]).copy()
    df = df.div(df.sum(axis=1), axis=0)

    df.index = df.index.astype(str)
    df.columns = df.columns.astype(str)
    df = df.reset_index()
    df = df.melt(id_vars='region_sample_donor')
    df = df.rename(columns={'value': 'perc', 'level2': 'celltype'})
    if condition:
        df["condition"] = condition
    
    return df


#### prepare variables for plot

In [12]:
# Order for regions and conditions
cond_order = ["Naive","Sham","MCAo"]
#region_order = ["Calvaria","Vertebra","Scapula","Humerus","Pelvis","Femur"]
region_sample_donor_order = [
    "Calvaria_1.1","Calvaria_1.2","Calvaria_1.3","Calvaria_2.1","Calvaria_2.2","Calvaria_2.3",
    "Vertebra_1.1","Vertebra_1.2","Vertebra_1.3","Vertebra_2.1","Vertebra_2.2","Vertebra_2.3",
    "Scapula_1.1","Scapula_1.2","Scapula_1.3","Scapula_2.1","Scapula_2.2","Scapula_2.3",
    "Humerus_1.1","Humerus_1.2","Humerus_1.3","Humerus_2.1","Humerus_2.2","Humerus_2.3",
    "Pelvis_1.1","Pelvis_1.2","Pelvis_1.3","Pelvis_2.1","Pelvis_2.2","Pelvis_2.3",
    "Femur_1.1","Femur_1.2","Femur_1.3","Femur_2.1","Femur_2.2","Femur_2.3",
]
#ct_order = [
#    'hematopoietic stem cell', 'common myeloid progenitor', 'granulocyte-monocyte progenitor', 'neutrophil-primed GMP', 
#    'monocyte-primed GMP', 'monocyte-DC progenitor (MDP)', 'common DC progenitor (CDP)', 'pro neutrophil', 'pre neutrophil', 
#    'immature neutrophil', 'mature neutrophil', 'monocyte progenitor', 'classical monocyte', 'non-classical monocyte', 
#    'pro B cell', 'pre B cell', 'immature B cell', 'mature B cell', 'plasma cell', 'Cd8 T cell', 'Cd4 T cell', 'gdT cell', 
#    'NK-T cell', 'NK cell', 'plasmacytoid DC', 'conventional DC1', 'conventional DC2', 'monocyte-derived DC', 'B cell-DC hybrid', 
#    'neutrophil-DC hybrid', 'perivascular macrophage', 'macrophage', 'antigen-presenting macrophage', 'basophil progenitor', 
#    'basophil', 'fibroblast', 'dural fibroblast', 'endothelial cell', 'Gnb3+ cell', 'megakaryocyte', 'innate lymphoid cell', 
#    'meningeal-Choroid Plexus cell', 'oligodendrocyte', 'adipose-derived stromal cell'
#    
#    #'progenitors', 'neutrophil', 'monocyte', 'B cell', 'T cell', 'NK cell',
#    #'NK-T cell', 'dendritic cell', 'macrophage', #'microglia',
#    #'basophil', 'structural cell', #'erythroid precursor', 'erythroid cell', 
#    #'brain cell', 'megakaryocyte', 'innate lymphoid cell'
#]
#ct_colors = adata.uns["level2_colors"].tolist()

ct_order = {
    'neutrophil': ['pro neutrophil','pre neutrophil','immature neutrophil','mature neutrophil'],
     'B cell': ['pro B cell','pre B cell','immature B cell','mature B cell','plasma cell'],
     'monocyte': ['monocyte progenitor','classical monocyte','non-classical monocyte'],
     'progenitors': ['hematopoietic stem cell','common myeloid progenitor','granulocyte-monocyte progenitor',
                     'neutrophil-primed GMP','monocyte-primed GMP','monocyte-DC progenitor (MDP)'],
     'basophil': ['basophil progenitor','basophil'],
     'dendritic cell': ['plasmacytoid DC','conventional DC1','conventional DC2','monocyte-derived DC',
                        'common DC progenitor (CDP)','B cell-DC hybrid','neutrophil-DC hybrid'],
     #'NK cell': ['NK cell'],
     'T cell': ['Cd8 T cell', 'gdT cell','Cd4 T cell'],
     #'innate lymphoid cell': ['innate lymphoid cell'],
     #'NK-T cell': ['NK-T cell'],
     'structural cell': ['fibroblast','dural fibroblast','adipose-derived stromal cell','endothelial cell',
                         'Gnb3+ cell','meningeal-Choroid Plexus cell'],
     'macrophage': ['perivascular macrophage','antigen-presenting macrophage','macrophage'],
     #'megakaryocyte': ['megakaryocyte'],
     #'brain cell': ['oligodendrocyte']
}

# after updating annotations not all cell types are still there (throw out accordingly)
for ct, cts2 in ct_order.items():
    ct_order[ct] = [c for c in cts2 if c in adata.obs["level2"].cat.categories]
    
all_ct_colors = adata.uns["level2_colors"].tolist()
all_cts = adata.obs["level2"].cat.categories.tolist()
ct_colors = {}
for ct1, ct2s in ct_order.items():
    ct_colors[ct1] = [all_ct_colors[all_cts.index(ct)] for ct in ct2s]


df_per_level1 = {}

for ct1, ct2s in ct_order.items():
    # Get df
    #bones = ['Femur', 'Humerus', 'Pelvis', 'Scapula', 'Skull','Vertebra']
    dfs = [
        get_proportions_df(adata,condition="Naive",celltypes=ct2s),
        get_proportions_df(adata,condition="Sham",celltypes=ct2s),
        get_proportions_df(adata,condition="MCAO",celltypes=ct2s),
    ]
    
    df = pd.concat(dfs)
    df.loc[df["condition"]=="MCAO","condition"] = "MCAo"
    #df.loc[df["region"]=="Skull","region"] = "Calvaria"
    
    df["condition"] = df["condition"].astype("category")
    df["condition"] = df["condition"].cat.reorder_categories(cond_order)
    #df["region"] = df["region"].astype("category")
    #df["region"] = df["region"].cat.reorder_categories(region_order)
    df["region_sample_donor"] = df["region_sample_donor"].astype("category")
    df["region_sample_donor"] = df["region_sample_donor"].cat.reorder_categories(
        [rsd for rsd in region_sample_donor_order if rsd in df["region_sample_donor"].cat.categories]
    )
    df["celltype"] = df["celltype"].astype("category")
    df["celltype"] = df["celltype"].cat.reorder_categories(ct_order[ct1])#[::-1])
    df["perc"] *= 100
    
    df_per_level1[ct1] = df

/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args, **kwargs)
/opt/python/lib/python3.8/site-packages/pandas/core/arrays/categorical.py:2487: FutureWarning: The `inplace` parameter in pandas.Categorical.remove_unused_categories is deprecated and will be removed in a future version.
  res = method(*args

### neutrohpils

In [13]:
CT = "neutrophil"
df_ = df_per_level1[CT]
ct_colors_ = ct_colors[CT]

In [14]:
%%R -i df_ -i ct_colors_

#font sizes
S_SIZE = 17
M_SIZE = 19
L_SIZE = 21

#figure size
WIDTH=10#4.33
HEIGTH=10

# Labels
X_LABEL = 'Region_Sample.Donor'
Y_LABEL = 'Cell proportion'
LEGEND_LABEL = 'Cell types'

options(repr.plot.width=1, repr.plot.height=1)
g1 <- ggplot(df_, aes(x=region_sample_donor, y=perc, fill=celltype)) +
    scale_fill_manual(values = ct_colors_) +
    geom_bar(position='stack',stat='identity')  +  
    labs(x=X_LABEL, y=Y_LABEL) +
    facet_grid(~condition, scales="free", space="free_x") +
    theme_classic() + 
    guides(fill=FALSE)+#guide_legend(title=LEGEND_LABEL,keyheight=0.8,keywidth=0.8,default.unit="cm")) +
    theme(strip.text = element_text(face="bold", size=L_SIZE),   # size of subtitles ("MCAO", "Naive")
          axis.title.x = element_text(face="bold", size=M_SIZE), # x axis label (Cell type)
          axis.title.y = element_text(face="bold", size=M_SIZE), # y axis label (Cell proportion)
          axis.text.x  = element_text(angle=90, hjust = 1, vjust =0.5,  size=S_SIZE-6), # x axis ticks (regions)
          axis.text.y = element_text(size=S_SIZE), # y axis tick labels (cell proportion percentage)
          legend.title = element_text(size=M_SIZE),
          legend.text = element_text(size=S_SIZE))
g1
ggsave(filename = paste0('./figures/stacked_proportions_donor_deconv_neutrophil.pdf'), 
      plot = g1, width = WIDTH, height=HEIGTH)

### B cells

In [15]:
CT = "B cell"
df_ = df_per_level1[CT]
ct_colors_ = ct_colors[CT]

In [16]:
%%R -i df_ -i ct_colors_

#font sizes
S_SIZE = 17
M_SIZE = 19
L_SIZE = 21

#figure size
WIDTH=10#4.33
HEIGTH=10

# Labels
X_LABEL = 'Region_Sample.Donor'
Y_LABEL = 'Cell proportion'
LEGEND_LABEL = 'Cell types'

options(repr.plot.width=1, repr.plot.height=1)
g1 <- ggplot(df_, aes(x=region_sample_donor, y=perc, fill=celltype)) +
    scale_fill_manual(values = ct_colors_) +
    geom_bar(position='stack',stat='identity')  +  
    labs(x=X_LABEL, y=Y_LABEL) +
    facet_grid(~condition, scales="free", space="free_x") +
    theme_classic() + 
    guides(fill=FALSE)+#guide_legend(title=LEGEND_LABEL,keyheight=0.8,keywidth=0.8,default.unit="cm")) +
    theme(strip.text = element_text(face="bold", size=L_SIZE),   # size of subtitles ("MCAO", "Naive")
          axis.title.x = element_text(face="bold", size=M_SIZE), # x axis label (Cell type)
          axis.title.y = element_text(face="bold", size=M_SIZE), # y axis label (Cell proportion)
          axis.text.x  = element_text(angle=90, hjust = 1, vjust =0.5,  size=S_SIZE-6), # x axis ticks (regions)
          axis.text.y = element_text(size=S_SIZE), # y axis tick labels (cell proportion percentage)
          legend.title = element_text(size=M_SIZE),
          legend.text = element_text(size=S_SIZE))
g1
ggsave(filename = paste0('./figures/stacked_proportions_donor_deconv_B cell.pdf'), 
      plot = g1, width = WIDTH, height=HEIGTH)

# stacked bar plots with alternative x tick labels

I am ashamed of the code below that produces (partially) the x tick labels that I wanted to achieve. R is not my favorite programming language.

In [67]:
region_sample_donor_to_sample = {
    'Calvaria_1.1'  :"1",
    'Calvaria_1.2'  :"2",
    'Calvaria_1.3'  :"3",
    'Femur_1.1'     :"1",
    'Femur_1.2'     :"2",
    'Femur_1.3'     :"3",
    'Humerus_1.1'   :"1",
    'Humerus_1.2'   :"2",
    'Humerus_1.3'   :"3",
    'Pelvis_1.1'    :"1",
    'Pelvis_1.2'    :"2",
    'Pelvis_1.3'    :"3",
    'Scapula_1.1'   :"1",
    'Scapula_1.2'   :"2",
    'Scapula_1.3'   :"3",
    'Vertebra_1.1'  :"1",
    'Vertebra_1.2'  :"2",
    'Vertebra_1.3'  :"3",
    'Calvaria_2.1'  :"4",
    'Calvaria_2.2'  :"5",
    'Calvaria_2.3'  :"6",
    'Femur_2.1'     :"4",
    'Femur_2.2'     :"5",
    'Femur_2.3'     :"6",
    'Humerus_2.1'   :"4",
    'Humerus_2.2'   :"5",
    'Humerus_2.3'   :"6",
    'Pelvis_2.1'    :"4",
    'Pelvis_2.2'    :"5",
    'Pelvis_2.3'    :"6",
    'Scapula_2.1'   :"4",
    'Scapula_2.2'   :"5",
    'Scapula_2.3'   :"6",
    'Vertebra_2.1'  :"4",
    'Vertebra_2.2'  :"5",
    'Vertebra_2.3'  :"6",
}
spaces = {'Calvaria':0, 'Femur':1, 'Humerus':2, 'Pelvis':3, 'Scapula':4, 'Vertebra':5}
for bone_idx in region_sample_donor_to_sample:
    bone = bone_idx.split("_")[0]
    space = "".join([" " for i in range(spaces[bone])])
    region_sample_donor_to_sample[bone_idx] = space + region_sample_donor_to_sample[bone_idx] + space #.rjust(1+spaces[bone])

df["sample"] = df["region_sample_donor"].apply(lambda s: region_sample_donor_to_sample[s])
df["sample"] = df["sample"].astype("category")

#### coarse
To see the plot example comment out `ggsave`. Note however that the figure size differs between jupyter output and saved pdf

In [69]:
%%R -i df -i ct_colors

#font sizes
S_SIZE = 17
M_SIZE = 19
L_SIZE = 21

#figure size
WIDTH=10#4.33
HEIGTH=10

# Labels
X_LABEL = 'Sample'
Y_LABEL = 'Cell proportion'
LEGEND_LABEL = 'Cell types'
#LABELS = c(
#    #Naive
#    '1','2','3'#,'1','2','3','1','2','3','1','2','3','1','2','3','1','2','3',
#    ##Sham
#    #'1','2','3','1','2','3','1','2','3','1','2','3','1','2','3','1','2','3',
#    ##MCAO
#    #'1','2','3','4','5','6','1','2','3','4','5','6','1','2','3','4','5','6',
#    #'1','2','3','4','5','6','1','2','3','4','5','6','1','2','3','4','5','6'
#)

options(repr.plot.width=1, repr.plot.height=1)
g1 <- ggplot(df, aes(x=sample, y=perc, fill=celltype)) + # x=region_sample_donor
    scale_fill_manual(values = ct_colors) +
    geom_bar(position='stack',stat='identity') +  
    labs(x=X_LABEL, y=Y_LABEL) +
    facet_grid(~condition, scales="free", space="free_x") + 
    theme_classic() + 
    #scale_x_discrete(labels=LABELS) + #breaks=0:35,  #breaks=BREAKS, 
    #scale_x_discrete(labels = df$sample) + #aes(labels = x), 
    guides(fill=FALSE)+#guide_legend(title=LEGEND_LABEL,keyheight=0.8,keywidth=0.8,default.unit="cm")) +
    theme(strip.text = element_text(face="bold", size=L_SIZE),   # size of subtitles ("MCAO", "Naive")
          axis.title.x = element_text(face="bold", size=M_SIZE), # x axis label (Cell type)
          axis.title.y = element_text(face="bold", size=M_SIZE), # y axis label (Cell proportion)
          #axis.text.x  = element_text(angle=90, hjust = 1, vjust =0.5,  size=S_SIZE-6), # x axis ticks (regions)
          axis.text.x  = element_text(angle=0, hjust = 0.5, vjust =0.5,  size=S_SIZE-2), # x axis ticks (regions)
          axis.text.y = element_text(size=S_SIZE), # y axis tick labels (cell proportion percentage)
          legend.title = element_text(size=M_SIZE),
          legend.text = element_text(size=S_SIZE))
g1
ggsave(filename = paste0('./figures/stacked_proportions_donor_deconv.pdf'), 
      plot = g1, width = WIDTH, height=HEIGTH)

### neutrohpils

In [76]:
CT = "neutrophil"
df_ = df_per_level1[CT]
df_["sample"] = df_["region_sample_donor"].apply(lambda s: region_sample_donor_to_sample[s])
df_["sample"] = df_["sample"].astype("category")
ct_colors_ = ct_colors[CT]

In [77]:
%%R -i df_ -i ct_colors_

#font sizes
S_SIZE = 17
M_SIZE = 19
L_SIZE = 21

#figure size
WIDTH=10#4.33
HEIGTH=10

# Labels
X_LABEL = 'Sample'
Y_LABEL = 'Cell proportion'
LEGEND_LABEL = 'Cell types'

options(repr.plot.width=1, repr.plot.height=1)
g1 <- ggplot(df_, aes(x=sample, y=perc, fill=celltype)) +
    scale_fill_manual(values = ct_colors_) +
    geom_bar(position='stack',stat='identity')  +  
    labs(x=X_LABEL, y=Y_LABEL) +
    facet_grid(~condition, scales="free", space="free_x") +
    theme_classic() + 
    guides(fill=FALSE)+#guide_legend(title=LEGEND_LABEL,keyheight=0.8,keywidth=0.8,default.unit="cm")) +
    theme(strip.text = element_text(face="bold", size=L_SIZE),   # size of subtitles ("MCAO", "Naive")
          axis.title.x = element_text(face="bold", size=M_SIZE), # x axis label (Cell type)
          axis.title.y = element_text(face="bold", size=M_SIZE), # y axis label (Cell proportion)
          axis.text.x  = element_text(angle=0, hjust = 0.5, vjust =0.5,  size=S_SIZE-2), # x axis ticks (regions)
          axis.text.y = element_text(size=S_SIZE), # y axis tick labels (cell proportion percentage)
          legend.title = element_text(size=M_SIZE),
          legend.text = element_text(size=S_SIZE))
g1
ggsave(filename = paste0('./figures/stacked_proportions_donor_deconv_neutrophil.pdf'), 
      plot = g1, width = WIDTH, height=HEIGTH)

### B cells

In [81]:
CT = "B cell"
df_ = df_per_level1[CT]
df_["sample"] = df_["region_sample_donor"].apply(lambda s: region_sample_donor_to_sample[s])
df_["sample"] = df_["sample"].astype("category")
ct_colors_ = ct_colors[CT]

In [82]:
%%R -i df_ -i ct_colors_

#font sizes
S_SIZE = 17
M_SIZE = 19
L_SIZE = 21

#figure size
WIDTH=10#4.33
HEIGTH=10

# Labels
X_LABEL = 'Sample'
Y_LABEL = 'Cell proportion'
LEGEND_LABEL = 'Cell types'

options(repr.plot.width=1, repr.plot.height=1)
g1 <- ggplot(df_, aes(x=sample, y=perc, fill=celltype)) +
    scale_fill_manual(values = ct_colors_) +
    geom_bar(position='stack',stat='identity')  +  
    labs(x=X_LABEL, y=Y_LABEL) +
    facet_grid(~condition, scales="free", space="free_x") +
    theme_classic() + 
    guides(fill=FALSE)+#guide_legend(title=LEGEND_LABEL,keyheight=0.8,keywidth=0.8,default.unit="cm")) +
    theme(strip.text = element_text(face="bold", size=L_SIZE),   # size of subtitles ("MCAO", "Naive")
          axis.title.x = element_text(face="bold", size=M_SIZE), # x axis label (Cell type)
          axis.title.y = element_text(face="bold", size=M_SIZE), # y axis label (Cell proportion)
          axis.text.x  = element_text(angle=0, hjust = 0.5, vjust =0.5,  size=S_SIZE-2), # x axis ticks (regions)
          axis.text.y = element_text(size=S_SIZE), # y axis tick labels (cell proportion percentage)
          legend.title = element_text(size=M_SIZE),
          legend.text = element_text(size=S_SIZE))
g1
ggsave(filename = paste0('./figures/stacked_proportions_donor_deconv_B cell.pdf'), 
      plot = g1, width = WIDTH, height=HEIGTH)